<a href="https://colab.research.google.com/github/SangHee33/Sanghee2/blob/main/%ED%95%B8%EC%A6%88%EC%98%A8_7%EC%9E%A5_%EC%95%99%EC%83%81%EB%B8%94_%ED%95%99%EC%8A%B5%EA%B3%BC_%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

p. 245-271

- 랜덤 포레스트 : 결정 트리의 앙상블

# 7.1 투표 기반 분류기
- 직접 투표 분류기 : 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측
- 개별 분류기 중 가장 뛰어난 것보다 정확도가 높은 경우가 많음
- 각 분류기가 약한 학습기라도 충분하게 많고 다양하다면 앙상블은 강한 학습기일 수 있음 -> 큰 수의 법칙 때문
- 모든 분류기가 완벽하게 독립적이고 오차에 상관관계가 없어야함

In [19]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [20]:
# 여러 분류기를 조합하여 사이킷런의 투표 기반 분류기 생성, 훈련
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [21]:
# 각 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.95
SVC 0.95
VotingClassifier 0.95


-> 투표 기반 분류기가 다른 개별 분류기보다 성능이 조금 더 높음

- 간접 투표 : 모든 분류기가 클래스의 확률을 예측할 수 있으면(predict_proba() 메서드가 있으면) 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스 예측
- voting="hard" 에서 "soft"로 변경
- SVC는 기본값에서 클래스 확률을 제공하지 않으므로 probability 매개변수를 True로 지정

# 7.2 배깅과 페이스팅

- 배깅 : 훈련 세트에서 중복을 허용하여 샘플링하는 방식
- 페이스팅 : 중복을 허용하지 않고 샘플링하는 방식
- 수집 함수는 전형적으로 분류일 때는 통계적 최빈값이고 회귀에 대해서는 평균 계산

## 7.2.1 사이킷런의 배깅과 페이스팅

In [22]:
# 결정 트리 분류기 500개의 앙상블 훈련
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

- 앙상블은 비슷한 편향에서 더 작은 분산을 만듦
- 배깅이 페이스팅보다 다양성이 크가 편향이 조금 더 높음 -> 예측기들의 상관관계를 줄이므로 앙상블의 분산 감소
- 시간과 CPU 파워에 여유가 있다면 교차 검증으로 배깅과 페이스팅을 모두 평가하여 더 나은 쪽 선택

## 7.2.2 oob 평가
- BaggingClassifier에서 선택되지 않은 훈련 샘플의 나머지 37%를 oob 샘플이라고 부름
- 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻음

In [23]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9

In [24]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9

In [25]:
# oob 샘플에 대한 결정 함수의 값은 oob_decision_function_ 변수에서 확인
bag_clf.oob_decision_function_

array([[0.01685393, 0.98314607],
       [0.43654822, 0.56345178],
       [0.55900621, 0.44099379],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.98395722, 0.01604278],
       [0.        , 1.        ],
       [0.00497512, 0.99502488],
       [1.        , 0.        ],
       [0.30978261, 0.69021739],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.93220339, 0.06779661],
       [0.00540541, 0.99459459],
       [0.43961353, 0.56038647],
       [0.5625    , 0.4375    ],
       [0.77894737, 0.22105263],
       [1.        , 0.        ],
       [0.98830409, 0.01169591],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.00543478, 0.99456522],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.2568306 , 0.7431694 ],
       [0.        , 1.        ],
       [0.98757764, 0.01242236],
       [0.        , 1.        ],
       [0.

# 7.3 랜덤 패치와 랜덤 서브스페이스
- 특성에 대한 샘플링
- 랜덤 패치 방식 : 훈련 특성과 샘플을 모두 샘플링하는 것
- 랜덤 서브스페이스 방식 : 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것
- 특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춤

# 7.4 랜덤 포레스트


In [26]:
# Bagging Classifier에 DecisionTreeClassifier 대신 RandomForestClassifier 사용
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [27]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

## 7.4.1 엑스트라 트리
- 익스트림 랜덤 트리(엑스트라 트리) : 극단적으로 무작위한 트리의 랜덤 포레스트
- 편향이 늘어나지만 분산이 낮아짐

## 7.4.2 특성 중요도
- 랜덤 포레스트는 상대적 중요도를 측정하기 쉬움
- 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도 측정
- 가중치 평균이며 각 노드의 가중치는 연관된 훈련 샘플 수와 같음

In [28]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.096084059788306
sepal width (cm) 0.02235854665118346
petal length (cm) 0.44168754116615516
petal width (cm) 0.43986985239435544


# 7.5 부스팅
- 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법
- 앞의 모델을 보완해나가면서 일련의 예측기 학습
- 에이다부스트와 그레이디언트 부스팅

## 7.5.1 에이다부스트
- 과소적합했던 훈련 샘플의 가중치를 더 높임
- 첫 번째 부류기를 훈련 세트에서 훈련시키고 예측
- 알고리즘이 잘못 분류된 훈련 셈플의 가중치를 상대적으로 높임
- 두 번째 분류기는 업데이트된 가중치를 사용해 훈련 세트에서 훈련하고 다시 예측
- 이 과정 반복
- 사이킷런은 SAMME라는 에이다부스트의 다중 클래스 버전을 사용

In [29]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

## 7.5.2 그레이디언트 부스팅
- 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가
- 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습

In [30]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [31]:
# 첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor 훈련
y2 = y-tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [32]:
y3 = y2-tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [ ]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [34]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

- learning_rate 매개변수가 각 트리의 기여 정도를 조절
- 낮게 설정하면 앙상블 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능은 좋아짐 -> 축소

In [35]:
# 최적의 트리 수를 찾기 위한 조기 종료 기법
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train,y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=22)

In [36]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up=0
  else:
    error_going_up += 1
    if error_going_up ==5:
      break # 조기종료

- 확률적 그레이디언트 부스팅 : subsample을 낮추어 편향이 높아지는 대신 분산을 낮추고 훈련 속도를 높임

In [37]:
# 최정화된 그레이디언트 부스팅
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [39]:
# 자동 조기 종료
xgb_reg.fit(X_train, y_train,
            eval_set =[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.39301
[1]	validation_0-rmse:0.31009
[2]	validation_0-rmse:0.25765
[3]	validation_0-rmse:0.22661
[4]	validation_0-rmse:0.19959
[5]	validation_0-rmse:0.18293
[6]	validation_0-rmse:0.17584
[7]	validation_0-rmse:0.16919
[8]	validation_0-rmse:0.16555
[9]	validation_0-rmse:0.16791
[10]	validation_0-rmse:0.16605


# 7.6 스태킹
- 예측기들은 각각 다른 값을 예측하고 마지막 예측기(블레더 또는 메타 학습기)가 이 예측을 입력으로 받아 최종 예측을 만듦